## Carga de datos, almacenamiento y formatos de archivo

In [1]:
import pandas as pd
import numpy as np

### Lectura y escritura de datos en formato de texto
Pandas presenta una serie de funciones para leer datos tabulares como un objeto
DataFrame. `pandas.read_csv` es uno de los más utilizados.

**Indexación**: puede tratar una o más columnas como el DataFrame devuelto, y si desea
obtener nombres de columna del archivo, argumentos que proporcione o no.

**Inferencia de tipo y conversión de datos**: incluye las conversiones de valor
definidas por el usuario y lista personalizada de marcadores de valores faltantes.

**Análisis de fecha y hora**: incluye una capacidad de combinación, que incluye la
combinación de información de fecha y hora distribuida en varias columnas en una sola
columna en el resultado.

**Iterando**: soporte para iterar sobre trozos de archivos muy grandes.

**Problemas de datos impuros**: incluye omitir filas o pie de página, comentarios u
otras cosas menores como datos numéricos con miles separados por coma.

In [2]:
!cat ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [3]:
df = pd.read_csv("ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
# un archivo no siempre tendrá una fila de encabezados
!cat ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [5]:
# dejar que pandas asigne nombre a las columnas
pd.read_csv("ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
# asignar nombre a las columnas
pd.read_csv("ex2.csv", names=list("abcd"))

,a,b,c,d
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [9]:
# indicar que la columna d sea índice
names = ["a", "b", "c", "d", "message"]
pd.read_csv("ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [10]:
"""
formar un índice jerárquico: pasar una lista de números
o nombres
"""
!cat csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [11]:
parsed = pd.read_csv("csv_mindex.csv",
                    index_col=["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

Un archivo no simpre puede tener un delimitador fijo:

In [12]:
# el archivo tiene una cantidad variable de espacios en blanco
!cat ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [13]:
# usar una expresión regular junto a pd.read_csv
result = pd.read_csv("ex3.txt", sep="\s+")
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Omitir la primera, tercera y cuarta fila de un archivo con `skiprow`:

In [14]:
!cat ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [16]:
pd.read_csv("ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


El manejo de valores faltantes es una parte importante y frecuentemente matizado del
proceso de lectura de archivos. Los datos faltantes generalmente no están presentes
(cadenas vacías) o marcados por algunos centinelas (placeholder). Por defecto, pandas
usa un conjunto de centinelas comunes, como `NA` y `NULL`:

In [17]:
!cat ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [18]:
result = pd.read_csv("ex5.csv")
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [19]:
pd.isna(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [48]:
result = pd.read_csv("ex5.csv", na_values=["NULL"])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


`keep_default_na` deshabilita la representación predeterminada para los `NA`:

In [22]:
result2 = pd.read_csv("ex5.csv", keep_default_na=False)
result2

,something,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [23]:
result2.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [24]:
result3 = pd.read_csv("ex5.csv", keep_default_na=False,
                     na_values=["NA"])
result3

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [25]:
result3.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [29]:
# especificar diferentes centinelas NA para cada columna en un diccionario
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
pd.read_csv("ex5.csv", na_values=sentinels,
           keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


### Lectura de archivos de texto en piezas

Antes de mirar un archivo grande, hacemos que la configuración de visualización de
pandas sea más compacta:

In [33]:
pd.options.display.max_rows = 10

In [34]:
result = pd.read_csv("ex6.csv")
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


Las marcas de elipsis `...` indican que se han omitido filas en medio del DataFrame.

In [36]:
# leer un pequeño número de filas con nrows
pd.read_csv("ex6.csv", nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [37]:
# leer un archivo en pedazos con chunksize
chunker = pd.read_csv("ex6.csv", chunksize=1000)
type(chunker)

pandas.io.parsers.readers.TextFileReader

`TextFileReader` devuelto por `pd.read_csv` le permite iterar sobre las partes del
archivo de acuerdo con `chunksize`. Por ejemplo, podemos iterar sobre `ex5.csv`,
agregando los recuentos de valor en la columna `"key"`:

In [38]:
chunker = pd.read_csv("ex6.csv", chunksize=1000)
tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)

In [39]:
# resultado
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Escribir datos de formato de texto
Los datos también se pueden exportar a un formato delimitado.

In [43]:
data = pd.read_csv("ex5.csv")
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Usando el método `to_csv`, podemos escribir los datos en un archivo separado por comas:

In [44]:
data.to_csv("out.csv")
!cat out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [45]:
import sys

data.to_csv(sys.stdout, sep="|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Los valores faltantes aparecen como cadenas vacías en la salida. Es posibe que desee
denotarlos por algún otro valor centinela:

In [46]:
data.to_csv(sys.stdout, na_rep="NULL")

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [49]:
!cat out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


Sin otras opciones especificadas, se escriben las etiquetas de filas y columna.
Ambos pueden ser deshabilitados:

In [51]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


### Trabajando con otros formatos delimitados
Es posible cargar la mayoría de las formas de datos tabulares desde el disco utilizando
funciones como `pd.read_csv`. En algunos casos, sin embargo, puede ser necesario un
procesamiento manueal. No es raro recibir un archivo con una o más líneas malformadas.

In [52]:
!cat ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


Para cualquier archivo con un delimitadr de un solo carácter, puede usar el módulo `csv`
incorporado de Python.

In [53]:
import csv

f = open("ex7.csv")

reader = csv.reader(f)

Al esterilizar a través del lector como un archivo produce listas de valores con
cualquier carácter de comillas eliminado:

In [54]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [55]:
f.close()

A partir de ahí, depende de usted hacer las disputas necesarias para poner los datos en
la forma que necesita. Ejemplo:

In [58]:
with open("ex7.csv") as f:
    lines = list(csv.reader(f))
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [59]:
# dividir las líneas para encabezado y las líneas de datos
header, values = lines[0], lines[1:]

In [66]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

Los archivos CSV vienen en muchos sabores diferentes. Para definir un nuevo formato con
un delimitador diferente, una convención de cotización de cadena o terminador de línea,
podríamos definir una subclase simple de `csv.Dialect`:

In [67]:
class MyDialect(csv.Dialect):
    lineterminator = "\n"
    delimiter = ";"
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [70]:
f = open("ex7.csv")
reader = csv.reader(f, dialect=MyDialect)

También pasar parámetros de dialectos individuales a CSV como palabras clave para
`csv.reader` sin tener que definir una subclase:

In [72]:
reader = csv.reader(f, delimiter="|")

Para escribir archivos delimitados manualmente se puede usar `csv.writer`.

In [73]:
with open("mydata.csv", "w") as f:
    writer = csv.writer(f, dialect=MyDialect)
    writer.writerow(("one", "two", "three"))
    writer.writerow(("1", "2", "3"))
    writer.writerow(("4", "5", "6"))
    writer.writerow(("7", "8", "9"))

In [74]:
!cat mydata.csv

one;two;three
1;2;3
4;5;6
7;8;9
